![dvd_image](dvd_image.jpg)

A DVD rental company needs your help! They want to figure out how many days a customer will rent a DVD for based on some features and has approached you for help. They want you to try out some regression models which will help predict the number of days a customer will rent a DVD for. The company wants a model which yeilds a MSE of 3 or less on a test set. The model you make will help the company become more efficient inventory planning.

The data they provided is in the csv file `rental_info.csv`. It has the following features:
- `"rental_date"`: The date (and time) the customer rents the DVD.
- `"return_date"`: The date (and time) the customer returns the DVD.
- `"amount"`: The amount paid by the customer for renting the DVD.
- `"amount_2"`: The square of `"amount"`.
- `"rental_rate"`: The rate at which the DVD is rented for.
- `"rental_rate_2"`: The square of `"rental_rate"`.
- `"release_year"`: The year the movie being rented was released.
- `"length"`: Lenght of the movie being rented, in minuites.
- `"length_2"`: The square of `"length"`.
- `"replacement_cost"`: The amount it will cost the company to replace the DVD.
- `"special_features"`: Any special features, for example trailers/deleted scenes that the DVD also has.
- `"NC-17"`, `"PG"`, `"PG-13"`, `"R"`: These columns are dummy variables of the rating of the movie. It takes the value 1 if the move is rated as the column name and 0 otherwise. For your convinience, the reference dummy has already been dropped.

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV

# Import any additional modules and start coding below
rental_data = pd.read_csv("rental_info.csv")
display(rental_data)
print(rental_data.dtypes)

rental_data["rental_date"] = pd.to_datetime(rental_data["rental_date"])
rental_data["return_date"] = pd.to_datetime(rental_data["return_date"])

rental_data["rental_length"] = pd.to_datetime(rental_data["return_date"]) - pd.to_datetime(rental_data["rental_date"])
rental_data["rental_length_days"] = rental_data["rental_length"].dt.days

rental_data["deleted_scenes"] = np.where(rental_data["special_features"].str.contains("Deleted Scenes"), 1,0)
rental_data["behind_the_scenes"] = np.where(rental_data["special_features"].str.contains("Behind the Scenes"), 1,0)
display(rental_data.head())

cols_to_drop = ["rental_date","return_date", "rental_length_days", "special_features", "rental_length"]

### Split data to feature and target sets
X = rental_data.drop(cols_to_drop, axis = 1)
y = rental_data["rental_length_days"]

### Split data to train test values

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = 9)

### Create Lasso model to pick most significant features
lasso = Lasso(alpha=0.3, random_state=9)

### Train model and access the coefficients
lasso.fit(X_train, y_train)
lasso_coef = lasso.coef_
print(lasso_coef)

### Perform feature selection by choosing column with positive coefficients
X_lasso_train, X_lasso_test = X_train.iloc[:, lasso_coef >0], X_test.iloc[:, lasso_coef > 0]

### Predictions by Linear Model
ols = LinearRegression()
ols = ols.fit(X_lasso_train, y_train)
y_test_pred = ols.predict(X_lasso_test)
mse_lin_reg_lasso = mean_squared_error(y_test, y_test_pred)
print(mse_lin_reg_lasso)

### Random forest hyperparameter range
param_dist = {
    "n_estimators": np.arange(1,101,1),
    "max_depth": np.arange(1,11,1)
}

### Create random forest regressor 
rf = RandomForestRegressor()

### Use random search to find the best hyperparameters for model
rand_search = RandomizedSearchCV(rf, param_distributions=param_dist, cv = 5, random_state=9)

### Fit the random search object to the data
rand_search.fit(X_train, y_train)

### Create a variable for the best hyperparameters
best_parameters = rand_search.best_params_
print(best_parameters)

### Run the random forest with best parameters

rf = RandomForestRegressor(n_estimators=best_parameters["n_estimators"],
                          max_depth=best_parameters["max_depth"],
                          random_state=9)
rf.fit(X_train, y_train)
rf_pred = rf.predict(X_test)
mse_random_forest = mean_squared_error(y_test, rf_pred)
print(mse_random_forest)
                    
best_model = rf
best_mse = mse_random_forest


rental_date  ... rental_rate_2
0      2005-05-25 02:54:33+00:00  ...        8.9401
1      2005-06-15 23:19:16+00:00  ...        8.9401
2      2005-07-10 04:27:45+00:00  ...        8.9401
3      2005-07-31 12:06:41+00:00  ...        8.9401
4      2005-08-19 12:30:04+00:00  ...        8.9401
...                          ...  ...           ...
15856  2005-08-22 10:49:15+00:00  ...       24.9001
15857  2005-07-31 09:48:49+00:00  ...       24.9001
15858  2005-08-20 10:35:30+00:00  ...       24.9001
15859  2005-07-31 13:10:20+00:00  ...       24.9001
15860  2005-08-18 06:33:55+00:00  ...       24.9001

[15861 rows x 15 columns]

rental_date          object
return_date          object
amount              float64
release_year        float64
rental_rate         float64
length              float64
replacement_cost    float64
special_features     object
NC-17                 int64
PG                    int64
PG-13                 int64
R                     int64
amount_2            float64
length_2            float64
rental_rate_2       float64
dtype: object


rental_date  ... behind_the_scenes
0 2005-05-25 02:54:33+00:00  ...                 1
1 2005-06-15 23:19:16+00:00  ...                 1
2 2005-07-10 04:27:45+00:00  ...                 1
3 2005-07-31 12:06:41+00:00  ...                 1
4 2005-08-19 12:30:04+00:00  ...                 1

[5 rows x 19 columns]

[ 5.84104424e-01  0.00000000e+00 -0.00000000e+00  0.00000000e+00
 -0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
 -0.00000000e+00  4.36220109e-02  3.01167812e-06 -1.52983561e-01
 -0.00000000e+00  0.00000000e+00]
4.812297241276244
{'n_estimators': 51, 'max_depth': 10}
2.225667528098759
